In [5]:
import numpy as np
def Matrix(NH, H, P, NL, NU):
    # Initialize constants and arrays
    BN = 2.5426228e-05  # cm-1/kgauss

    global nlb, lb

    # Define array R (3x3x3)
    R = np.array([
        [1.1449056155209570e-01, 0.0, 0.0],
        [0.0, 3.8075110222036970e-02, -5.4033883822915113e-02],
        [0.0, -5.4033883822915113e-02, -2.4122750777996062e-01],
        [1.2247448713915891e+00, 0.0, 0.0],
        [0.0, 3.2659863237109041e+00, 5.7735026918962577e-01],
        [0.0, 5.7735026918962577e-01, -2.0655911179772890e+00],
        [0.0, 0.0, 0.0],
        [0.0, 0.0, 1.0],
        [0.0, -1.0, 1.0]]).reshape(3, 3, 3)

    W = np.zeros(5)
    QI = np.zeros(5)
    QJ = np.zeros(4)

    # Initializing W(4) and W(5)
    W[3] = math.sqrt(P[8] * (P[8] + 1.0) * (2.0 * P[8] + 1.0))
    W[4] = math.sqrt(0.2 * (P[8] + 1.0) * (2.0 * P[8] + 1.0) * (2.0 * P[8] + 3.0) / P[8] / (2.0 * P[8] - 1.0))
    QI[4] = P[8]

    # Main calculation of the Hamiltonian matrix elements for block MJ + MI
    for I in range(NL, NU + 1):  # from NL to NU
        IR = int(I - NL + 1)
        QI[0], QI[1], QI[2], QI[3] = list(map(float, lb[I - 1][0:16].split()))
        IRD = int(QI[0] + QI[1] + 0.5)
        for J in range(I, NU + 1):
            IC = int(J - NL + 1)
            QJ[0], QJ[1], QJ[2], QJ[3] = list(map(float, lb[J - 1][0:16].split()))
            ICD = int(QJ[0] + QJ[1] + 0.5)
            H[IR - 1, IC - 1] = 0.0

            # Spin orbit interaction
            if I == J:
                if QI[0] == 0:
                    H[IR - 1, IR - 1] = P[5]
                elif QI[0] == 1:
                    H[IR - 1, IR - 1] = P[0] + 0.5 * P[1] * (QI[1] * (QI[1] + 1) - 2.75)

            # Electronic Zeeman interaction
            W[0] =THJ(float(QI[1]), 1.0,  float(QJ[1]), float(-QI[2]), float(QI[2]-QJ[2]), float(QJ[2]))

            if QI[3] == QJ[3]:
                N = QI[2] - 0.5
                W[2] = W[0]
                if N % 2 != 0:
                    W[2] = -W[2]
                H[IR - 1, IC - 1] += P[11] * W[2] * R[IRD - 1, ICD - 1, 0]

            # Nuclear hyperfine interaction
            W[1] = THJ(float(QI[4]), 1.0, float(QI[4]), float(-QI[3]), float(QI[3] - QJ[3]), float(QJ[3]))
            W[2] = W[0] * W[1] * W[3]
            N = QI[4] - QI[3] - QJ[2] + 0.5
            if N % 2 != 0:
                W[2] = -W[2]
            W[2] *= R[IRD - 1, ICD - 1, 1]

            if QI[0] == 0:
                H[IR - 1, IC - 1] += P[6] * W[2]
            if QI[0] == 1 and QI[1] == QJ[1] and QI[1] == 1.5:
                H[IR - 1, IC - 1] += (P[2] + P[3]) * W[2]
            if QI[0] == 1 and QI[1] == QJ[1] and QI[1] == 0.5:
                H[IR - 1, IC - 1] += (P[2] - P[3]) * W[2]
            if QI[0] == 1 and QI[1] != QJ[1]:
                H[IR - 1, IC - 1] += P[2] * W[2]

            # Electric quadrupole interaction
            W[0] = THJ(float(QI[1]), 2.0, float(QJ[1]), float(-QI[2]), float(QI[2] - QJ[2]), float(QJ[2]))
            W[2] = THJ(float(QI[4]), 2.0, float(QI[4]), float(-QI[3]), float(QI[3] - QJ[3]), float(QJ[3]))
            W[0] = W[0]* W[2] * W[4]
            N = QI[4] - QI[3] - QJ[2] - 0.5
            if N % 2 != 0:
                W[0] = -W[0]
            H[IR - 1, IC - 1] += P[4] * W[0] * R[IRD - 1, ICD - 1, 2]

            # Nuclear Zeeman interaction
            if QI[1] == QJ[1] or QI[2] == QJ[2]:
                W[0] = W[1] * W[3] / P[8]
                N = QI[4] - QI[3] + 1
                if N % 2 != 0:
                    W[0] = -W[0]
                H[IR - 1, IC - 1] += P[7] * P[11] * BN * W[0]

            # Set lower triangular half equal to the upper triangular half
            if I != J:
                H[IC - 1, IR - 1] = H[IR - 1, IC - 1]

    return NH, H


In [7]:
p=[14903.89, 0.223559, 1.6918e-4, -1.2882e-4, -3.6860e-7, 0.0, 0.0050747387200, 0.8220300, 1.0, 26.4, 26.4,
         0.0742]
H=np.zeros((8,8))
import math
Matrix(8,H,p,1,8)

NameError: name 'math' is not defined